In [139]:
import timm
import torch
from torch import nn
import torch.nn.functional as F

from apn import APN, decorrelation_loss, compactness_loss

%load_ext autoreload
%autoreload 2

In [128]:
net = APN(k=112)
x = torch.randn(4, 3, 224, 224)
global_logits, local_logits, attn_maps, prototypes, logit_idxs = net(x)
global_logits.shape, local_logits.shape, attn_maps.shape, prototypes.shape, logit_idxs.shape

In [129]:
dummy = torch.zeros(4, 7, 11, 11)
dummy[:, :, 3:-3, 3:-3] = 0.5
dummy[:, :, 5, 5] = 1


max_local_logits = torch.amax(dummy, dim=(-1, -2), keepdim=True)
max_logits_coords = torch.nonzero(dummy == max_local_logits)
max_logits_coords = max_logits_coords[..., 2:]
res = compactness_loss(dummy, max_logits_coords)
print(res)

dummy = torch.zeros(4, 7, 11, 11)
dummy[:, :, 1:-1, 1:-1] = 0.5
dummy[:, :, 5, 5] = 1

max_local_logits = torch.amax(dummy, dim=(-1, -2), keepdim=True)
max_logits_coords = torch.nonzero(dummy == max_local_logits)
max_logits_coords = max_logits_coords[..., 2:]
res = compactness_loss(dummy, max_logits_coords)
print(res)

dummy = torch.zeros(4, 7, 11, 11)
dummy[:, :, 4:-4, 3:-3] = torch.tensor(0.5)
dummy[:, :, 5, 5] = 1

max_local_logits = torch.amax(dummy, dim=(-1, -2), keepdim=True)
max_logits_coords = torch.nonzero(dummy == max_local_logits)
max_logits_coords = max_logits_coords[..., 2:]
res = compactness_loss(dummy, max_logits_coords)
print(res)

In [163]:
dummy1 = torch.tensor([0, 1] * 10, dtype=torch.float32)
dummy2 = torch.tensor([1, 0] * 10, dtype=torch.float32)
dummy = torch.stack([dummy1] * 5 + [dummy2] * 5, dim=0)* 2
print(dummy.to(torch.long))
decorrelation_loss(dummy, group_idxs=torch.tensor([0,0,0,0,0,1,1,1,1,1]))

In [164]:
dummy1 = torch.tensor([1] * 4 + [0] * 16, dtype=torch.float32)
dummy2 = torch.tensor([0] * 4 + [1] * 4 + [0] * 12, dtype=torch.float32)
dummy3 = torch.tensor([0] * 8 + [1] * 4 + [0] * 8, dtype=torch.float32)
dummy4 = torch.tensor([0] * 12 + [1] * 4 + [0] * 4, dtype=torch.float32)
dummy5 = torch.tensor([0] * 16 + [1] * 4, dtype=torch.float32)
dummy = torch.stack([dummy1, dummy2, dummy3, dummy4, dummy5] * 2) * 2
print(dummy.to(torch.long))
decorrelation_loss(dummy, group_idxs=torch.tensor([0,0,0,0,0,1,1,1,1,1]))